# Introduction to pyjeo

**Material provided by Dr. Pieter Kempeneers (European Commission, Joint Research Centre)**

[pyjeo](https://pyjeo.readthedocs.io) is the follow up of [PKTOOLS](http://pktools.nongnu.org/html/index.html), a suite of utilities written in C++ for image processing with a focus on remote sensing applications. It is distributed under a General Public License (GPLv3) as a Python package.

In a nutshell, the main differences between pyjeo and pktools from a user's perspective are:

- pyjeo is a Python package should be run in a **Python environment**, whereas pktools applications are run from the command line (e.g., in a bash shell)
- pyjeo runs with images entirely **in memory**, whereas pktools runs most applications line per line. This makes pyjeo considerably faster, but with a larger memory footprint. However, there are some methods implemented in pyjeo to reduce the memory footprint by tiling the image


Run the following script to perform the installation of pyjeo

    sudo apt update
    sudo apt upgrade -y  
    cd ~/Downloads
    wget https://raw.githubusercontent.com/selvaje/SE_data/main/exercise/install_pyjeo4osgeo16.sh
    ./install_pyjeo4osgeo16.sh



Import Python modules

In [ ]:
# Trick to consider for venv within a notebook: simply use a clean sys.path taken from the shell
import sys
sys.path

In [ ]:
sys.path = ['', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '/home/user/venv/lib/python3.10/site-packages', '/home/user/.local/lib/python3.10/site-packages', '/home/user/.local/lib/python3.10/site-packages/pyjeo-1.1.2-py3.10.egg', '/home/user/.local/lib/python3.10/site-packages/rioxarray-0.15.5-py3.10.egg', '/home/user/.local/lib/python3.10/site-packages/pyproj-3.6.1-py3.10-linux-x86_64.egg', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3.10/dist-packages']

In [ ]:
sys.path

In [ ]:
# from now on all should work as expected...

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime
import pyjeo as pj

Call the inline help function for [pj.geometry.warp](https://pyjeo.readthedocs.io/en/latest/3_reference.html#geometry.warp)

In [ ]:
help(pj.geometry.warp)

## Jim: a geospatial raster dataset object

Open an image and show its properties

In [ ]:
jim = pj.Jim('geodata/vegetation/ETmean08-11.tif')

In [ ]:
jim.properties.nrOfCol()

A Jim object is a geospatial dataset within a coordinate reference system

In [ ]:
jim.properties.getProjection()

From [gdal](https://gdal.org/tutorials/geotransforms_tut.html#introduction-to-geotransforms): "A geotransform is an affine transformation from the image coordinate space (row, column), also known as (pixel, line) to the georeferenced coordinate space (projected or geographic coordinates)"

Geotransform is an array with 6 items:

[0] top left x

[1] w-e pixel resolution

[2] rotation, 0 if image is “north up”

[3] top left y

[4] rotation, 0 if image is “north up”

[5] n-s pixel resolution

In [ ]:
jim.properties.getGeoTransform()

We have a single band and single plane raster object

In [ ]:
jim.properties.nrOfBand()

In [ ]:
jim.properties.nrOfPlane()

Open a second GeoTIFF image of the same dimension

In [ ]:
gpp = pj.Jim('geodata/vegetation/GPPmean08-11.tif')

Stack this image to the existing bands of jim, creating a multi-band jim object

In [ ]:
jim.geometry.stackBand(gpp)

In [ ]:
jim.properties.nrOfBand()

## Exercise 1: stackBand function vs. method

Create a new jim object `jim_stacked` using the [stackBand](https://pyjeo.readthedocs.io/en/latest/3_reference.html#geometry.stackBand) function instead of the [stackBand](https://pyjeo.readthedocs.io/en/latest/3_reference.html#geometry._Geometry.stackBand) method. It should contain two bands, one for `ETmean08-11.tif` and one for `GPPmean08-11.tif`.

In [ ]:
jim_stacked = pj.geometry.stackBand(jim, gpp)

In [ ]:
jim_stacked.properties.nrOfBand()

## Labeled dimensions

Let's label the dimensions

In [ ]:
bandnames = ['ET', 'GPP']
planenames = [datetime.strptime('2019-08-11','%Y-%m-%d')]
jim.properties.setDimension({'band' : bandnames, 'plane' : planenames})

In [ ]:
jim.properties.getDimension('band')

In [ ]:
len(jim.properties.getDimension('plane'))

## Exercise 2: cropBand using numbered and labeled index

Crop second band as a new image (index starts from 0)

In [ ]:
jim1 = pj.geometry.cropBand(jim, 1)

Create a new jim object `gpp` that contains the second band using the [cropBand]() function and the labeled index ('GPP')

In [ ]:
gpp = pj.geometry.cropBand(jim, 'GPP')

Check if results are identical

In [ ]:
jim1.properties.isEqual(gpp)

In [ ]:
jim1.properties.getDimension()

## Working with multi-plane Jim objects

First convert the bands to planes using [band2plane](https://pyjeo.readthedocs.io/en/latest/3_reference.html#geometry._Geometry.band2plane)

In [ ]:
jim.geometry.band2plane()

The labels of band are copied to plane

In [ ]:
jim.properties.getDimension()

## Calculate composites via reducePlane

Reduce the planes by calculating the mean of all planes

In [ ]:
jim_mean = pj.geometry.reducePlane(jim, 'mean')

Where is the RuntimeWarning coming from?

In [ ]:
jim.stats.getStats()

In [ ]:
jim[jim < 0] = 0
jim.stats.getStats()

In [ ]:
jim_mean = pj.geometry.reducePlane(jim, 'mean')

### Calculate custom composites via callback function in reducePlane

In [ ]:
def getMax(reduced, plane):
    return pj.pixops.supremum(reduced, plane)
jim_max = pj.geometry.reducePlane(jim, getMax)

Crop the plane `ET` from `jim` using the method [cropPlane](https://pyjeo.readthedocs.io/en/latest/3_reference.html#geometry._Geometry.cropPlane). Check the resulting `jim` is a single plane and single band Jim object

In [ ]:
# original code by Pieter...
jim.geometry.cropPlane('ET')
print(jim.properties.nrOfPlane())
print(jim.properties.nrOfBand())

In [ ]:
# A little fix...
jim.geometry.cropPlane(0)
print(jim.properties.nrOfPlane())
print(jim.properties.nrOfBand())

In [ ]:
jim.geometry.plane2band()
bandnames = ['ET']
planenames = [datetime.strptime('2019-08-11','%Y-%m-%d')]
jim.properties.setDimension({'band' : bandnames, 'plane' : planenames})

## Bridging to third party libraries: Numpy

In [ ]:
jim.np()

Check the type of jim.np()

In [ ]:
type(jim.np())

Use [sum](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.sum.html#numpy-ndarray-sum) function of Numpy to assert there are no pixels where `jim_max < jim_mean`

In [ ]:
(jim_max < jim_mean).np().sum()

### Get items

With pyjeo we create the masks in memory in a "pythonic" way using [get items](https://jeodpp.jrc.ec.europa.eu/services/processing/pyjeohelp/3_reference.html#get-jim-items) without the need to write temporary files.

Get sub-dataset based on pixel coordinates (first 3 rows and columns, starting from 0)

In [ ]:
jim[0:3,0:3]

Last 3 rows and columns, show geographic bounding box [ulx, uly, lrx, lry]

In [ ]:
jim[-3:,-3:].properties.getBBox()

In [ ]:
jim[0:3,0:3].np()

### Set items

In pyjeo, we can apply the mask in a "pythonic" way using [set items](https://jeodpp.jrc.ec.europa.eu/services/processing/pyjeohelp/3_reference.html#set-jim-items)

In [ ]:
jim[0:3,0:3] = 0
jim[0:5,0:5].np()

**Display image using matplotlib**

We can show an image with matplotlib by providing a Numpy representation of the dataset

In [ ]:
# plt.gray()  # show the filtered result in grayscale
fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(111)
ax1.imshow(jim.np())
plt.show()

## Calculate median filter via scipy

In [ ]:
from scipy import ndimage
npimage = ndimage.median_filter(jim.np(), size = 5)

In [ ]:
# plt.gray()  # show the filtered result in grayscale
fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(111)
ax1.imshow(npimage)
plt.show()

## Exercise 4: Create a geospatial Jim object with CRS from a Numpy array

**A numpy array is not a geospatial dataset with a spatial coordinate reference system !**

In [ ]:
ajim = pj.np2jim(npimage)
print(ajim.properties.getProjection())
print(ajim.properties.getGeoTransform())

Use [setProjection](https://pyjeo.readthedocs.io/en/latest/3_reference.html#properties._Properties.setProjection) and [getProjection](https://pyjeo.readthedocs.io/en/latest/3_reference.html#properties._Properties.getProjection) to set the Coordinate Reference System (CRS) of ajim

## Calculating the median_filter in place using the output parameter

In [ ]:
ndimage.median_filter(jim.np(), output = jim.np(), size = 5)

As an alternative, we can set the Numpy representation of a Jim object in place

In [ ]:
jim.np()[:] = ndimage.median_filter(jim.np(), size = 5)

## Bridging to third party libraries: Xarray

In [ ]:
jim.xr()

**An Xarray dataset is defined with a spatial coordinate reference system !**

In [ ]:
ajim = pj.xr2jim(jim.xr())

In [ ]:
print(ajim.properties.getProjection())
print(ajim.properties.getGeoTransform())

At the basis we find a Numpy data array

In [ ]:
type(jim.xr().ET.data)

### Calculate median filter via Xarray

Processing can take several minutes...

In [ ]:
# xr_median = jim.xr().ET.rolling(x = 3, y = 3, center=True).median()

In [ ]:
# xr_median.plot()

**Notice that XArray member functions return the processed result and do not alter the input image !**

## Bridging JimVect to third party libraries

In [ ]:
sample = pj.JimVect('geodata/shp/polygons.sqlite')

In [ ]:
import pandas as pd
pdf = pd.DataFrame(sample.dict())
pdf

## Data extraction and regional statistics

In [ ]:
extracted = pj.geometry.extract(sample, jim, rule='mean', output='/vsimem/mean.json', oformat='GeoJSON')

In [ ]:
pdf = pd.DataFrame(extracted.dict())
pdf